In [1]:
from datasets.Reflections import Count
from notebooks.notebooks import get_ttv_paths
import torch
from transforms.transforms import autocorrelation

In [2]:
train_path, test_path, validate_path = get_ttv_paths()

train_dataset = Count(train_path, 'train', download=False, transform=autocorrelation)
test_dataset = Count(test_path, 'test', download=False, transform=autocorrelation)
validate_dataset = Count(validate_path, 'validate', download=False, transform=autocorrelation)

In [3]:
print(train_dataset[0])

(array([[-0.11828613, -0.11828613],
       [-0.38079834, -0.38079834],
       [-0.38858032, -0.38858032],
       [ 0.9373779 ,  0.9373779 ],
       [-0.5359192 , -0.5359192 ],
       [-0.23815918, -0.23815918],
       [-0.70635986, -0.70635986],
       [ 0.1274414 ,  0.1274414 ],
       [-0.9844971 , -0.9844971 ],
       [-0.37756348, -0.37756348],
       [-0.2930298 , -0.2930298 ],
       [ 0.00262451,  0.00262451],
       [-0.2899475 , -0.2899475 ],
       [ 0.6123352 ,  0.6123352 ],
       [ 0.01867676,  0.01867676],
       [-0.84628296, -0.84628296],
       [-0.29330444, -0.29330444],
       [ 0.02008057,  0.02008057],
       [-0.9746704 , -0.9746704 ],
       [-0.32226562, -0.32226562],
       [ 0.15539551,  0.15539551],
       [-0.64767456, -0.64767456],
       [ 0.9319458 ,  0.9319458 ],
       [ 0.4401245 ,  0.4401245 ],
       [ 0.1973877 ,  0.1973877 ],
       [-0.2602539 , -0.2602539 ],
       [-0.1003418 , -0.1003418 ],
       [-0.44796753, -0.44796753],
       [ 0.6759033 

In [4]:
device = torch.device("cuda")
print(f"Using {device}")

Using cuda


In [5]:
def train_network(train, test, validate):
    from torch.utils.data import DataLoader
    from torch import nn
    import torch
    from utils.neural_networks import test_loop, train_loop, validate_loop

    input_size = 1
    for v in train[0][0].shape:
        input_size *= v

    output_size = 9
    batch_size = 32

    learning_rate = 0.1
    epochs = 5

    train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True)
    validate_dataloader = DataLoader(validate, batch_size=batch_size, shuffle=True)

    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(input_size, 256),
                nn.ReLU(),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Dropout(p=0.25),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Linear(256, output_size)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to()
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        validate_loop(validate_dataloader, model, loss_fn)
        test_loop(test_dataloader, model, loss_fn)
    print("Done")

In [ ]:
train_network(train_dataset, test_dataset, validate_dataset)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=194, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.2, inplace=False)
    (5): Linear(in_features=256, out_features=256, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.25, inplace=False)
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.2, inplace=False)
    (11): Linear(in_features=256, out_features=9, bias=True)
  )
)
Epoch 1
-------------------------------
Accuracy: 21.88%, batch loss: 2.172204 [    0/ 6000]
Accuracy: 21.88%, batch loss: 2.059929 [  320/ 6000]
Accuracy: 18.75%, batch loss: 1.981529 [  640/ 6000]
Accuracy: 25.00%, batch loss: 1.898586 [  960/ 6000]
Accuracy: 18.75%, batch loss: 1.830761 [ 1280/ 6000]
Accuracy: 25.00%, batch loss: 1.769387 [ 1600/ 6000]
Accuracy: 15.62%, batch loss: 1.726486 [ 1920/ 6